In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [5]:

# Step 1: Load the model (Mistral-7B or Llama-2-7B)
def load_model():
    model_name = "mistralai/Mistral-7B-v0.1"  # Replace with "meta-llama/Llama-2-7b" if using Llama-2
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

# Step 2: Generate code using the model
def generate_code(model, tokenizer, user_request):
    prompt = f"Write Python code to: {user_request}"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs.input_ids, max_length=200, num_return_sequences=1)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# Step 3: Mock model for testing (if no GPU or model access)
def mock_generate_code(user_request):
    if "highest total transaction amount from the CSV" in user_request:
        return """
import pandas as pd

def find_top_spender(file_path):
    df = pd.read_csv(file_path)
    totals = df.groupby('user_id')['amount'].sum()
    top_spender = totals.idxmax()
    return top_spender, totals[top_spender]
"""
    elif "calculate the average of a list" in user_request:
        return """
def calculate_average(lst):
    return sum(lst) / len(lst)
"""
    else:
        return "I don't know how to handle this request."

# Step 4: Test the generated code
def test_code(code):
    try:
        # Execute the generated code
        exec(code, globals())
        print("Code executed successfully.")
        # Example: Test the find_top_spender function
        if "find_top_spender" in code:
            data = {'user_id': ['user101', 'user202', 'user101', 'user202'], 'amount': [100, 200, 130, 250.5]}
            df = pd.DataFrame(data)
            df.to_csv("transactions.csv", index=False)
            top_spender, total = find_top_spender("transactions.csv")
            print(f"Top spender: {top_spender}, Total amount: {total}")
        # Example: Test the calculate_average function
        elif "calculate_average" in code:
            lst = [1, 2, 3, 4, 5]
            avg = calculate_average(lst)
            print(f"Average of the list: {avg}")
    except Exception as e:
        print(f"Error during testing: {e}")

# Step 5: Main loop for the code assistant
def main():
    print("Welcome to the Code Assistant Agent!")
    use_mock = input("Do you want to use the mock model? (yes/no): ").lower() == "yes"

    if not use_mock:
        print("Loading the model...")
        model, tokenizer = load_model()
    else:
        print("Using the mock model.")

    while True:
        user_input = input("Enter your coding request (type 'exit' to quit): ")
        if user_input.lower() == "exit":
            break

        if use_mock:
            generated_code = mock_generate_code(user_input)
        else:
            generated_code = generate_code(model, tokenizer, user_input)

        print("\nGenerated Code:")
        print(generated_code)

        print("\nTesting the code...")
        test_code(generated_code)

if __name__ == "__main__":
    main()

Welcome to the Code Assistant Agent!
Loading the model...


KeyboardInterrupt: 

In [3]:
# Import libraries
from llama_index.core import Document
import openai

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader # Correct

In [5]:
from llama_index.core import GPTVectorStoreIndex

In [1]:
import os
os.environ["GROQ_API_KEY"] = "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"
import os
from groq import Groq

In [54]:

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write a function to add a row in a CSV file. write in python",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

**Adding a Row to a CSV File in Python**

Here's an example of a function that adds a row to a CSV file using the `csv` module in Python.

### Prerequisites

* Python 3.x
* `csv` module (part of the Python standard library)

### Code

```python
import csv

def add_row_to_csv(filename, row):
    """
    Add a row to a CSV file.

    Args:
        filename (str): The name of the CSV file.
        row (list): The row to add to the CSV file.

    Returns:
        None
    """
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)

# Example usage:
filename = 'example.csv'
row = ['John', 'Doe', '25']

# Create a new CSV file with a header row
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['First Name', 'Last Name', 'Age'])

# Add a row to the CSV file
add_row_to_csv(filename, row)

# Add another row to the CSV file
add_row_to_csv(filename, ['Jane', 'Doe', '30'])
```

### E

In [6]:
# Install necessary packages if not already installed
# pip install llama-index-llms-openai

from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

from llama_index.llms.openai import OpenAI

# Define custom events
class CodeStartEvent(StartEvent):
    request: str

class CodeGenerateEvent(Event):
    code: str

class CodeTestEvent(Event):
    result: str

# Define the workflow class
class CodeAssistantFlow(Workflow):
    llm = OpenAI(model="gpt-3.5-turbo")

    @step
    # async def process_request(self, ev: CodeStartEvent) -> CodeGenerateEvent:
    #     prompt = f"Write Python code to {ev.request}."
    #     response = await self.llm.acomplete(prompt)
    #     code = str(response)
    #     return CodeGenerateEvent(code=code)

    async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
        prompt = f"Write Python code to {ev.request}."
        response = await self.llm.acomplete(prompt)
        code = str(response)
        return CodeGenerateEvent(code=code)
    
    @step
    async def test_code(self, ev: CodeGenerateEvent) -> CodeTestEvent:
        # Simulate testing the generated code
        # For safety, return a placeholder result
        return CodeTestEvent(result="Code tested successfully.")

    @step
    async def provide_feedback(self, ev: CodeTestEvent) -> StopEvent:
        feedback = f"Test result: {ev.result}"
        return StopEvent(result=feedback)

# Run the workflow
if __name__ == "__main__":
    import asyncio

    w = CodeAssistantFlow(timeout=60, verbose=True)
    # result = asyncio.run(w.run(event=CodeStartEvent(request="Find the user with the highest total transaction amount from the CSV.")))
    result = asyncio.run(w.run(request="Find the user with the highest total transaction amount from the CSV."))
    print(str(result))

RuntimeError: asyncio.run() cannot be called from a running event loop

Running step generate_code


Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._achat in 1.0439344770825012 seconds as it raised APIConnectionError: Connection error..


In [7]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [35]:
# Install necessary packages if not already installed
# pip install llama-index groq

import os
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Define custom events
class CodeStartEvent(StartEvent):
    request: str

class CodeGenerateEvent(Event):
    code: str

class CodeTestEvent(Event):
    result: str

# Define the workflow class
class CodeAssistantFlow(Workflow):
    @step
    
    # async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
    #     prompt = f"Write Python code to {ev.request}."
    #     response = await self.llm.acomplete(prompt)
    #     code = str(response)
    #     return CodeGenerateEvent(code=code)
    
    
    

    async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
        # Use Groq API to generate code
        # chat_completion = client.chat.completions.create(
        #     messages=[
        #         {
        #             "role": "user",
        #             "content": f"Write Python code to {ev.request}.",
        #         }
        #     ],
        #     model="llama-3-70b-versatile",  # Use the appropriate Groq model
        # )
        
        chat_completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI-powered Python code generator. Your task is to generate clean, efficient, and well-commented Python code based on the user's request. Follow these guidelines:\n\nUnderstand the Task:\n\nCarefully interpret the user's programming request.\n\nIf the request is unclear, ask for clarification or make reasonable assumptions.\n\nGenerate Code:\n\nProvide Python code that solves the user's problem.\n\nInclude comments to explain key steps or logic.\n\nUse meaningful variable and function names for clarity.\n\nHandle potential edge cases and exceptions.\n\nCode Structure:\n\nOrganize the code logically, with functions or classes where appropriate.\n\nPrioritize readability and simplicity over unnecessary complexity.\n\nTesting Readiness:\n\nEnsure the code is ready to be tested or executed.\n\nIf the code requires user input, clearly indicate where and how the input should be provided.\n\nAssumptions:\n\nAssume all necessary libraries (e.g., pandas, numpy) are installed.\n\nIf the user does not specify details (e.g., file paths, data formats), make reasonable assumptions and state them clearly.\n\nError Handling:\n\nInclude basic error handling (e.g., try-except blocks) where applicable.\n\nInform the user of potential issues and how to resolve them.\n\nExample Output:\n\nProvide an example of how the code would work with sample data or inputs.\n\n"
        
                },
                {
                    "role": "user",
                    "content": f"Write Python code to {ev.request}.",
                },
            ],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        print(chat_completion)
        # k=""
        # for chunk in chat_completion:
        #     print(chunk.choices[0].delta.content or "", end="")
            
            
        j = ""
        # store the above printed message in a variable
        for chunk in chat_completion:
            k=chunk.choices[0].delta.content or ""
            # print(k)  
            j = j+k  
            
        #print the data type of k
        # print(type(k))
        
        print(j)
        
        
        # code = chat_completion.choices[0].message.content
        code = j
                
        # print(k)
        return CodeGenerateEvent(code=code)

    @step
    async def test_code(self, ev: CodeGenerateEvent) -> CodeTestEvent:
        # Simulate testing the generated code
        # For safety, return a placeholder result
        return CodeTestEvent(result="Code tested successfully.")

    @step
    async def provide_feedback(self, ev: CodeTestEvent) -> StopEvent:
        feedback = f"Test result: {ev.result}"
        return StopEvent(result=feedback)

# Run the workflow
if __name__ == "__main__":
    import asyncio

    w = CodeAssistantFlow(timeout=60, verbose=True)
    # result = asyncio.run(w.run(event=CodeStartEvent(request="Find the user with the highest total transaction amount from the CSV.")))
    # result = asyncio.run(w.run(request="Find the user with the highest total transaction amount from the CSV."))
    result = await w.run(request="Find the user with the highest total transaction amount from the CSV.")
    print(str(result))


Running step generate_code
Here's the Python code to find the user with the highest total transaction amount from a CSV file containing transaction data:

```python
import csv

def read_transaction_data(file_path):
    """
    Reads transaction data from a CSV file and returns a list of dictionaries.
    
    Each dictionary contains the transaction data for a single user, with
    keys 'user_id' and 'transaction_amount'.
    
Step generate_code produced event CodeGenerateEvent
Running step test_code
Step test_code produced event CodeTestEvent
Running step provide_feedback
Step provide_feedback produced event StopEvent
Test result: Code tested successfully.
